# Least-Squares Estimate of Cycling Parameters

In [3]:
import sys; sys.path.append('..')
import numpy as np
import fitparse
import pandas as pd
import math
import plotly.express as px

from utils.paths import data_folder

In [14]:
def parse_fitfile(
  filepath: str,
  columns: list[str],
  verbose: bool = False,
) -> pd.DataFrame:
  """Parse a .fit file into a format the simulator can use.

  Notes
  -----
  The file must include at least timestamp, distance, and altitude.
  """
  ff = fitparse.FitFile(filepath)
  df = {col: [] for col in columns}

  generator = ff.get_messages("record", with_definitions=True)
  messages = list(generator)

  for m in messages:
    if type(m) == fitparse.records.DefinitionMessage:
      continue

    record_data = m.get_values()

    if verbose:
      print(record_data.keys())

    if any([col not in record_data for col in df]):
      ts = record_data["timestamp"]
      if verbose:
        print(f"Skipping incomplete record @ {ts}")
      continue

    for col in df:
      df[col].append(record_data[col])

  return pd.DataFrame(df)


def preprocess_columns(df):
  """Preprocess grade."""
  df["dy"] = df.altitude.diff(1)
  df["dx"] = df.distance.diff(1)
  df["grade"] = (df.dy / df.dx).fillna(0)
  df["theta"] = df.grade.map(math.atan).fillna(0)
  df["v2"] = df.speed ** 2
  df["v2_diff"] = df.v2.diff(1).fillna(0)
  df["speed_diff"] = df.speed.diff(1).fillna(0)
  df["dt"] = df.timestamp.diff(1).fillna(0)
  df["P_prev_sec"] = df.power.shift(1).fillna(0)
  df.dt = df.dt.map(lambda x: x.total_seconds() if type(x) == pd.Timedelta else x)
  return df

In [100]:
def estimate_parameters(df: pd.DataFrame):
  G = 9.81 # m/s2
  m = 72 + 10 # kg
  rho = 1.225 # kg/m3

  dv = df.speed_diff.to_numpy()
  v0 = df.speed.to_numpy() - df.speed_diff.to_numpy()
  dv2 = df.v2_diff.to_numpy()
  dx = df.dx.to_numpy()
  dy = df.dy.to_numpy()
  dt = df.dt.to_numpy()
  P_legs = df.power.to_numpy()
  theta = df.theta.to_numpy()

  # Change in kinetic energy + change in potential energy from previous row to current row (t-1 -> t).
  b = 0.5*m*dv2 + m*G*dy
  c1 = dt * P_legs
  c2 = -dx * m * G * np.cos(theta)
  c3 = -0.5 * rho * dx * (v0**2 + v0*dv + (1/3)*dv**2)
  A = np.column_stack((c1, c2, c3))

  x, residuals, rank, s = np.linalg.lstsq(A, b)

  yhat = A @ x

  return x, residuals, rank, s, yhat, b


def estimate_parameters_constant_L(df: pd.DataFrame, L: float = 0.05):
  G = 9.81 # m/s2
  m = 72 + 10 # kg
  rho = 1.225 # kg/m3

  dv = df.speed_diff.to_numpy()
  v0 = df.speed.to_numpy() - df.speed_diff.to_numpy()
  dv2 = df.v2_diff.to_numpy()
  dx = df.dx.to_numpy()
  dy = df.dy.to_numpy()
  dt = df.dt.to_numpy()
  P_legs = df.power.to_numpy()
  theta = df.theta.to_numpy()

  # Change in kinetic energy + change in potential energy from previous row to current row (t-1 -> t).
  c1 = dt * P_legs
  b = 0.5*m*dv2 + m*G*dy - c1*(1-L)
  c2 = -dx * m * G * np.cos(theta)
  c3 = -0.5 * rho * dx * (v0**2 + v0*dv + (1/3)*dv**2)
  A = np.column_stack((c2, c3))

  x, residuals, rank, s = np.linalg.lstsq(A, b)

  yhat = A @ x

  return x, residuals, rank, s, yhat, b


def estimate_parameters_twostep(df: pd.DataFrame, verbose: bool = False):
  _, _, _, _, yhat, b = estimate_parameters(df)
  df["error"] = b - yhat
  df["abs_error"] = np.abs(df.error)
  before = len(df)

  df = df[df.abs_error < np.percentile(df.abs_error, 99)]

  if verbose:
    print(f"Removed {before - len(df)} outliers")

  x, residuals, rank, s, yhat, b = estimate_parameters(df)

  return x, residuals, rank, s, yhat, b, df

In [108]:
cols = [
  'timestamp',
  'distance',
  'altitude',
  'position_lat',
  'position_long',
  'power',
  'speed',
]
filepath = data_folder("fit/Boston_Tri.fit")
# filepath = data_folder("fit/Great_Brook_Farm.fit")
# filepath = data_folder("fit/IM_Santa_Cruz_70.3.fit")
df = parse_fitfile(filepath, cols, verbose=False)
df = preprocess_columns(df)
df.drop(labels=[0], inplace=True)

# Get rid of any rows where there is no change in time, or a long pause.
df = df[df.dt == 1]
# # Get rid of rows where speed is zero.
df = df[df.speed > 1]

# x, residuals, rank, s, y_hat, b = estimate_parameters(df)
# x, residuals, rank, s, y_hat, b, df2 = estimate_parameters_twostep(df, verbose=True)
# L = 1 - x[0]
# Crr = x[1]
# CdA = x[2]

chunk_size = 10

results = dict(Crr=[], CdA=[])
for i in range(0, len(df), chunk_size):
  _df = df[i:i+chunk_size]

  L = 0.05
  x, residuals, rank, s, y_hat, b = estimate_parameters_constant_L(_df, L=L)
  Crr = x[0]
  CdA = x[1]
  print(f"-- Optimized parameters:")
  print(f"* Crr = {Crr:.4f}")
  print(f"* CdA = {CdA:.4f}")
  results["Crr"].append(Crr)
  results["CdA"].append(CdA)
  # print(f"* Residuals = {residuals[0]:.4f}")
  # print(f"* Rank = {rank}")
  # print(f"* Singular values = {s}")

-- Optimized parameters:
* Crr = -0.3829
* CdA = 4.9619
-- Optimized parameters:
* Crr = -0.1163
* CdA = 1.8192
-- Optimized parameters:
* Crr = 0.0591
* CdA = -0.3398
-- Optimized parameters:
* Crr = 0.0641
* CdA = -0.4492
-- Optimized parameters:
* Crr = 0.0713
* CdA = -0.5395
-- Optimized parameters:
* Crr = -0.0513
* CdA = 0.9847
-- Optimized parameters:
* Crr = -0.1492
* CdA = 2.0760
-- Optimized parameters:
* Crr = -0.0044
* CdA = 0.3842
-- Optimized parameters:
* Crr = -0.7106
* CdA = 8.5969
-- Optimized parameters:
* Crr = 0.0314
* CdA = -0.0215
-- Optimized parameters:
* Crr = 0.0138
* CdA = 0.2274
-- Optimized parameters:
* Crr = -0.1501
* CdA = 2.0312
-- Optimized parameters:
* Crr = 0.1644
* CdA = -1.3993
-- Optimized parameters:
* Crr = 0.2902
* CdA = -3.1568
-- Optimized parameters:
* Crr = -0.1381
* CdA = 2.0728
-- Optimized parameters:
* Crr = -0.0965
* CdA = 1.6242
-- Optimized parameters:
* Crr = 0.4819
* CdA = -5.7079
-- Optimized parameters:
* Crr = 0.4413
* CdA = -

/var/folders/0z/fnhwmrr91f362mk6_bndszhw0000gn/T/ipykernel_28863/1679689537.py:50: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [109]:
fig = px.scatter(x=results["CdA"], y=results["Crr"])
fig.show()

In [48]:
fig = px.line(df, x="timestamp", y=["error"], title="Actual vs. Predicted Acceleration")
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 1, whereas the length of  previously-processed arguments ['timestamp'] is 8775

In [50]:
fig = px.line(df, x="timestamp", y=["altitude"], title="Variables")
fig.show()

In [ ]:
fig = px.histogram(df2, x="abs_error", title="Error Distribution")
fig.show()

np.percentile(df2.abs_error, 95)

In [51]:
def virtual_elevation(df: pd.DataFrame, m: float, CdA: float, Crr: float, L: float, rho: float = 1.225):
  G = 9.81 # m/s2

  dv = df.speed_diff.to_numpy()
  v0 = df.speed.to_numpy() - df.speed_diff.to_numpy()
  dv2 = df.v2_diff.to_numpy()
  dx = df.dx.to_numpy()
  dy = df.dy.to_numpy()
  dt = df.dt.to_numpy()
  P_legs = df.power.to_numpy()
  theta = df.theta.to_numpy()

  W_legs = P_legs * dt * (1 - L)
  W_roll = Crr * dx * m * G * np.cos(theta)
  W_air = 0.5 * rho * CdA * dx * (v0**2 + v0*dv + (1/3)*dv**2)

  delta_kinetic = 0.5 * m * dv2

  delta_h = (W_legs - W_roll - W_air - delta_kinetic) / (m * G)
  df["virtual_elevation"] = delta_h.cumsum() + df.altitude.iloc[0]

  return df


In [91]:
cols = [
  'timestamp',
  'distance',
  'altitude',
  'position_lat',
  'position_long',
  'power',
  'speed',
]
filepath = data_folder("fit/Boston_Tri.fit")
# filepath = data_folder("fit/Great_Brook_Farm.fit")
# filepath = data_folder("fit/IM_Santa_Cruz_70.3.fit")
df = parse_fitfile(filepath, cols, verbose=False)
df = preprocess_columns(df)
df.drop(labels=[0], inplace=True)

m = 72 + 10 # kg
rho = 1.225 # kg/m3
CdA = 0.30
Crr = 0.006
L = 0.05
df = virtual_elevation(df, m, CdA, Crr, L, rho)
df

,timestamp,distance,altitude,position_lat,position_long,power,speed,dy,dx,grade,theta,v2,v2_diff,speed_diff,dt,P_prev_sec,virtual_elevation
1,2023-08-27 11:49:54,432.69,9.4,504987797,-847636027,333,10.545,0.2,10.53,0.018993,0.018991,111.197025,3.328029,0.159,1.0,305.0,9.297020
2,2023-08-27 11:49:55,443.25,9.4,504988760,-847635306,354,10.537,0.0,10.56,0.000000,0.000000,111.028369,-0.168656,-0.008,1.0,333.0,9.392298
3,2023-08-27 11:49:56,453.73,9.4,504989752,-847634498,325,10.515,0.0,10.48,0.000000,0.000000,110.565225,-0.463144,-0.022,1.0,354.0,9.471605
4,2023-08-27 11:49:57,464.27,9.4,504990713,-847633678,306,10.373,0.0,10.54,0.000000,0.000000,107.599129,-2.966096,-0.142,1.0,325.0,9.658302
5,2023-08-27 11:49:58,474.55,9.6,504991659,-847632949,256,10.387,0.2,10.28,0.019455,0.019453,107.889769,0.290640,0.014,1.0,306.0,9.631143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,2023-08-27 12:40:49,32763.37,7.4,504936313,-847643406,375,7.722,0.0,7.69,0.000000,0.000000,59.629284,11.936448,0.816,1.0,227.0,-30.370720
3057,2023-08-27 12:40:50,32771.25,7.2,504935639,-847642683,401,7.811,-0.2,7.88,-0.025381,-0.025375,61.011721,1.382437,0.089,1.0,375.0,-30.123449
3058,2023-08-27 12:40:51,32779.38,7.2,504934989,-847641945,403,8.095,0.0,8.13,0.000000,0.000000,65.529025,4.517304,0.284,1.0,401.0,-30.044010
3059,2023-08-27 12:40:52,32787.59,7.0,504934349,-847641128,386,7.955,-0.2,8.21,-0.024361,-0.024356,63.282025,-2.247000,-0.140,1.0,403.0,-29.643651


In [92]:
fig = px.line(df, x="timestamp", y=["altitude", "virtual_elevation"], title="Actual vs. Virtual Elevation")
fig.show()